In [1]:
from flask import Flask, render_template, request
import os
import pickle
import numpy as np
from scipy.stats import poisson,skellam
import pandas as pd
from datetime import date
from update import update_model
from apiRequest import getFutureFixture

In [2]:
predictor = pickle.load(open(os.path.join('predictor/plk_objects/predictor.plk'), 'rb'))

In [3]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

In [4]:
future_fixtures = getFutureFixture()

In [5]:
future_fixtures

[['2020-02-22T04:30:00-08:00', 'Chelsea', 'Tottenham'],
 ['2020-02-22T07:00:00-08:00', 'Burnley', 'Bournemouth'],
 ['2020-02-22T07:00:00-08:00', 'Crystal Palace', 'Newcastle'],
 ['2020-02-22T07:00:00-08:00', 'Sheffield Utd', 'Brighton'],
 ['2020-02-22T07:00:00-08:00', 'Southampton', 'Aston Villa'],
 ['2020-02-22T09:30:00-08:00', 'Leicester', 'Manchester City'],
 ['2020-02-23T06:00:00-08:00', 'Manchester United', 'Watford'],
 ['2020-02-23T06:00:00-08:00', 'Wolves', 'Norwich'],
 ['2020-02-23T08:30:00-08:00', 'Arsenal', 'Everton'],
 ['2020-02-24T12:00:00-08:00', 'Liverpool', 'West Ham']]

In [6]:
def changeClubName(matches):

    for match in matches:
        home = match[1]
        away = match[2]

        if match[1] == 'Manchester City':
            match[1] = 'Man City'
        elif match[1] == 'Manchester United':
            match[1] = 'Man United'
        elif match[1] == 'Sheffield Utd':
            match[1] = 'Sheffield United'

        if match[2] == 'Manchester City':
            match[2] = 'Man City'
        elif match[2] == 'Manchester United':
            match[2] = 'Man United'
        elif match[2] == 'Sheffield Utd':
            match[2] = 'Sheffield United'
            
    return matches

upcoming_fixtures = changeClubName(future_fixtures)

In [7]:
upcoming_fixtures

[['2020-02-22T04:30:00-08:00', 'Chelsea', 'Tottenham'],
 ['2020-02-22T07:00:00-08:00', 'Burnley', 'Bournemouth'],
 ['2020-02-22T07:00:00-08:00', 'Crystal Palace', 'Newcastle'],
 ['2020-02-22T07:00:00-08:00', 'Sheffield United', 'Brighton'],
 ['2020-02-22T07:00:00-08:00', 'Southampton', 'Aston Villa'],
 ['2020-02-22T09:30:00-08:00', 'Leicester', 'Man City'],
 ['2020-02-23T06:00:00-08:00', 'Man United', 'Watford'],
 ['2020-02-23T06:00:00-08:00', 'Wolves', 'Norwich'],
 ['2020-02-23T08:30:00-08:00', 'Arsenal', 'Everton'],
 ['2020-02-24T12:00:00-08:00', 'Liverpool', 'West Ham']]

In [9]:
home_prop = []
draw_prop = []
away_prop = []

for index in upcoming_fixtures:
    home = index[1]
    away = index[2]
    result = simulate_match(predictor, home, away, max_goals=10)
    home_prop.append(np.sum(np.tril(result, -1)))
    draw_prop.append(np.sum(np.diag(result)))
    away_prop.append(np.sum(np.triu(result, 1)))

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
print(predictor)

In [10]:
result = simulate_match(predictor, home, away, max_goals=10)

AttributeError: 'NoneType' object has no attribute 'predict'

In [12]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import pickle


epl_1920 = pd.read_csv("http://football-data.co.uk/mmz4281/1920/E0.csv")
epl_1920 = epl_1920[['HomeTeam','AwayTeam','FTHG','FTAG','FTR']]
epl_1920 = epl_1920.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals','FTR':'Result'})


goal_model_data = pd.concat([epl_1920[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           epl_1920[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, family=sm.families.Poisson()).fit()

In [13]:
poisson_model